In [1]:
import json
import pandas as pd
import numpy as np
import requests 
import os
import sys
import pickle
from scipy import stats
from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web
from yahoo_fin import stock_info as si
from performance_analysis import annualized_return
from performance_analysis import annualized_standard_deviation
from performance_analysis import max_drawdown
from performance_analysis import gain_to_pain_ratio
from performance_analysis import calmar_ratio
from performance_analysis import sharpe_ratio
from performance_analysis import sortino_ratio
import yfinance as yf
import apiclient.discovery
from oauth2client.service_account import ServiceAccountCredentials
import httplib2
from sklearn import mixture as mix
import seaborn as sns 
import gspread as gd
import statsmodels.api as sm

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# подключение к гугл таблице

LIST = 'Сингапур'

index = '^AXJO'

exchange = 'SGX:'

exchange_yahoo =  '.SI'

# tickers = si.tickers_sp500()
cheked_year = '2015' 
cheked_year_end = '2020' 


# Файл, полученный в Google Developer Console
CREDENTIALS_FILE = 'Seetzzz-1cb93f64d8d7.json'
# ID Google Sheets документа (можно взять из его URL)
spreadsheet_id = '1lDhu6-tBmoh66a1mY3RU2yPV2_3uIzNSQWNI5UtMcag'
spreadsheet_id2 = '1A3leW6ZfsoVEPXZsv0Loj4eAbyKRchnHrJLdP4RIXDA'
#
# Авторизуемся и получаем service — экземпляр доступа к API
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    CREDENTIALS_FILE,
    ['https://www.googleapis.com/auth/spreadsheets',
     'https://www.googleapis.com/auth/drive'])
httpAuth = credentials.authorize(httplib2.Http())
service = apiclient.discovery.build('sheets', 'v4', http=httpAuth)

# ____________________________Парсим тикеры !!!!С ТАБЛИЦЫ!!!! и работаем с ними _______________________________________

# Чтения файла
values = service.spreadsheets().values().get(
    spreadsheetId=spreadsheet_id,
    range=f'{LIST}!A1:AA1000',
    majorDimension='COLUMNS'
).execute()

tickers = values['values'][12][2:202]

print(tickers)

['SGX:K3PD', 'SGX:K3OD', 'SGX:D05', 'SGX:O39', 'SGX:K6S', 'SGX:U11', 'SGX:J36', 'SGX:K3SD', 'SGX:Z77', 'SGX:Z74', 'SGX:F34', 'SGX:K3ED', 'SGX:S07', 'SGX:C31', 'SGX:K3ID', 'SGX:K3RD', 'SGX:Y92', 'SGX:Q0F', 'SGX:C6L', 'SGX:C38U', 'SGX:A17U', 'SGX:S63', 'SGX:K3HD', 'SGX:S68', 'SGX:G07', 'SGX:H78', 'SGX:BN4', 'SGX:BVA', 'SGX:G13', 'SGX:M44U', 'SGX:C07', 'SGX:ME8U', 'SGX:N2IU', 'SGX:C09', 'SGX:U14', 'SGX:BS6', 'SGX:V03', 'SGX:K3FD', 'SGX:BUOU', 'SGX:K3DD', 'SGX:AZY', 'SGX:D01', 'SGX:O32', 'SGX:S58', 'SGX:AJBU', 'SGX:K71U', 'SGX:T82U', 'SGX:T15', 'SGX:J69U', 'SGX:MZH', 'SGX:STG', 'SGX:U06', 'SGX:CJLU', 'SGX:U96', 'SGX:RW0U', 'SGX:C52', 'SGX:TQ5', 'SGX:HMN', 'SGX:HKB', 'SGX:T39', 'SGX:E5H', 'SGX:H02', 'SGX:C2PU', 'SGX:T14', 'SGX:A7RU', 'SGX:BSL', 'SGX:NC2', 'SGX:M04', 'SGX:SK6U', 'SGX:AIY', 'SGX:S59', 'SGX:A50', 'SGX:OV8', 'SGX:TS0U', 'SGX:EB5', 'SGX:CC3', 'SGX:Z25', 'SGX:F99', 'SGX:NS8U', 'SGX:AU8U', 'SGX:H13', 'SGX:AP4', 'SGX:F17', 'SGX:J91U', 'SGX:H15', 'SGX:CY6U', 'SGX:UD2', 'SGX:S08', 'S

# Читаем данные из огурцов

In [3]:
# # ['annuals']

Data_for_Portfolio_TOTAL = pd.DataFrame()
 
for ticker in tickers:
    with open(f'''C:/Users/Anton/Desktop/Backtesting/BANKA/data_json_{ticker.replace(exchange, '')}.pickle''', 'rb') as f:
        data_json = pickle.load(f)

    with open(f'''C:/Users/Anton/Desktop/Backtesting/BANKA/data_json_keyratios_{ticker.replace(exchange, '')}.pickle''', 'rb') as f:
        data_json_keyratios = pickle.load(f)

#     print(data_json)    
    try:
        date_list = pd.Series(data_json['financials']['annuals']['Fiscal Year'])
        keyratios = pd.DataFrame(data_json_keyratios['Fundamental'], index=[0]) 
        income_df = pd.DataFrame(data_json['financials']['annuals']['income_statement']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('-', 0).replace('', 0).replace('N/A', 0).astype(float)
        balance_df = pd.DataFrame(data_json['financials']['annuals']['balance_sheet']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('', 0).replace('-', 0).replace('N/A', 0).astype(float)
        cashflow_df = pd.DataFrame(data_json['financials']['annuals']['cashflow_statement']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('', 0).replace('-', 0).replace('N/A', 0).astype(float)
        valuation_ratios_df = pd.DataFrame(data_json['financials']['annuals']['valuation_ratios']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('', 0).replace('-', 0).replace('N/A', 0).astype(float)
        valuation_and_quality_df = pd.DataFrame(data_json['financials']['annuals']['valuation_and_quality']).set_index(date_list).drop(['Restated Filing Date', 'Filing Date', 'Earnings Release Date'], axis=1).replace('', 0).replace('No Debt', 0).replace('At Loss', 0).replace('-', 0).replace('N/A', 0).astype(float)
        common_size_ratios_df = pd.DataFrame(data_json['financials']['annuals']['common_size_ratios']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('', 0).replace('-', 0).replace('N/A', 0).replace('Negative Tangible Equity', 0).astype(float)
        # per_share_data_array_df = pd.DataFrame(data_json['financials']['annuals']['per_share_data_array']).set_index(date_list).replace('-', 0).replace('N/A', 0).astype(float)
        per_share_data_df = pd.DataFrame(data_json['financials']['annuals']['per_share_data_array']).set_index(date_list).replace('', 0).replace('No Debt', 0).replace('-', 0).replace('N/A', 0).astype(float)
        
        check = 1
    except:
        check = 0
        print(ticker)
        
        pass
            
    if check == 1:
    
        try:

            Data_for_Portfolio = pd.DataFrame()
        #     
            Data_for_Portfolio['E/P'] = income_df['Net Income'] / valuation_and_quality_df['Market Cap'] 
            # income_df['Net Income']
            # valuation_and_quality_df.replace('-', 0)


    #             Data_for_Portfolio['EBITDA/EV'] = income_df['EBITDA'] / (valuation_and_quality_df['Enterprise Value ($M)']*1000000)
            try:
                Data_for_Portfolio['EBITDA/EV'] = 1/valuation_ratios_df['EV-to-EBITDA']
            except:
                Data_for_Portfolio['EBITDA/EV'] = income_df['Pretax Income'] / valuation_and_quality_df['Enterprise Value ($M)']



            Data_for_Portfolio['ncfcommon'] = cashflow_df['Free Cash Flow']/(valuation_and_quality_df['Shares Outstanding (EOP)']**1000)

    #             try:    
    #                 Data_for_Portfolio['Total Debt'] = (balance_df['Short-Term Debt'] + balance_df['Long-Term Debt'])
            Data_for_Portfolio['Total Debt'] =per_share_data_df['Total Debt per Share']*(valuation_and_quality_df['Shares Outstanding (EOP)']*1000)
    #             except:


            Data_for_Portfolio['Shareholder Yield'] = -((Data_for_Portfolio['Total Debt'] + cashflow_df['Free Cash Flow'] \
                                             + Data_for_Portfolio['ncfcommon']) / valuation_and_quality_df['Market Cap'])


            Data_for_Portfolio['FCF/P'] = cashflow_df['Free Cash Flow'] / valuation_and_quality_df['Market Cap']
            Data_for_Portfolio
            # Data_for_Portfolio['Shareholder Yield'] 
            Data_for_Portfolio['Book Value per Share'] = per_share_data_df['Book Value per Share']
            Data_for_Portfolio['Dividends per Share'] =per_share_data_df['Dividends per Share'] 
            Data_for_Portfolio['Dividend Payout Ratio'] =common_size_ratios_df['Dividend Payout Ratio']

            try:
                Data_for_Portfolio['FCF/Assets'] = cashflow_df['Free Cash Flow'] / balance_df['Total Current Assets']
            except:
                Data_for_Portfolio['FCF/Assets'] = cashflow_df['Free Cash Flow'] / (balance_df['Balance Statement Cash and cash equivalents'] + balance_df['Accounts Receivable'])                                   

            #Can you generate returns on investment?   



            Data_for_Portfolio['ROA'] = common_size_ratios_df['ROA %']  
            Data_for_Portfolio['ROIC'] = common_size_ratios_df['ROIC %']
            #Do you have a defendable business model?

            try:
                Data_for_Portfolio['GROSS MARGIN'] = common_size_ratios_df['Gross Margin %']

            except:
                Data_for_Portfolio['GROSS MARGIN'] = common_size_ratios_df['Net Interest Margin (Bank Only) %']


            try:
                Data_for_Portfolio['CURRENT RATIO'] = valuation_and_quality_df['Current Ratio'] 
            except:
                Data_for_Portfolio['CURRENT RATIO'] = 1/common_size_ratios_df['Debt-to-Equity']

            try:
                Data_for_Portfolio['INTEREST/EBITDA'] = income_df['Interest Expense'] / income_df['EBITDA']
            except:
                Data_for_Portfolio['INTEREST/EBITDA'] = income_df['Interest Expense'] / income_df['Pretax Income']

#             print('tut')    
   
                
    # balance_df['Total Equity']
    # per_share_data_df['EPS without NRI']
    # income_df['Revenue']
    # income_df['Net Margin %']

            total_equity_grows_list = []
            EPS_grows_list = []
            rvenue_grows_list = []

        #  (500-400)/400*100=25%


            total_equity_grows_list.append(0)
            EPS_grows_list.append(0)
            rvenue_grows_list.append(0)

            for year in range(len(Data_for_Portfolio)):

                try:
                    total_equity_grows_list.append((balance_df['Total Equity'][year+1] - balance_df['Total Equity'][year])/ balance_df['Total Equity'][year]*100)
                except:
                    pass
                try:
                    EPS_grows_list.append((per_share_data_df['EPS without NRI'][year+1] - per_share_data_df['EPS without NRI'][year])/ per_share_data_df['EPS without NRI'][year]*100)
                except:
                    pass
                try:
                    rvenue_grows_list.append((income_df['Revenue'][year+1] - income_df['Revenue'][year])/ income_df['Revenue'][year]*100)
                except:
                    pass

                
                
            mean_total_equity_grows_list = [0, 0, 0, 0]
            mean_EPS_grows_list = [0, 0, 0, 0]
            mean_rvenue_grows_list = [0, 0, 0, 0]
            margin_params_list = [0, 0, 0, 0]

            for yearzz in range(len(Data_for_Portfolio)):
                if len(total_equity_grows_list[yearzz:5+yearzz])  == 5:
                    mean_total_equity_grows_list.append(np.mean(total_equity_grows_list[yearzz:5+yearzz]))
                else:
                    pass

                if len(EPS_grows_list[yearzz:5+yearzz])  == 5:
                    mean_EPS_grows_list.append(np.mean(EPS_grows_list[yearzz:5+yearzz]))
                else:
                    pass

                if len(rvenue_grows_list[yearzz:5+yearzz])  == 5:
                    mean_rvenue_grows_list.append(np.mean(rvenue_grows_list[yearzz:5+yearzz]))
                else:
                    pass

            Data_for_Portfolio['Net Margin %'] = income_df['Net Margin %']   


             
            
            for k in range(len(Data_for_Portfolio)):
    #             print(len(Data_for_Portfolio['Net Margin %'][k:5+k]))
                if len(Data_for_Portfolio['Net Margin %'][k:5+k])  == 5:
                    Y = Data_for_Portfolio['Net Margin %'][k:5+k].astype(float)
                    X = list(range(len(date_list[k:5+k])))
                    # X = sm.add_constant(X)
                    model = sm.OLS(Y,X)
                    results = model.fit()
                    margin_params_list.append(results.params[0])
    #                 print(results.params[0])
                else:
                    pass

    #         print(margin_params_list)
    #         print(Data_for_Portfolio['Net Margin %'])

#             print('tut22')
    
    
            Data_for_Portfolio['Total Equity Grows 5Y'] = mean_total_equity_grows_list
            Data_for_Portfolio['EPS without NRI Grows 5Y'] = mean_EPS_grows_list
            Data_for_Portfolio['Revenue Grows 5Y'] = mean_rvenue_grows_list
            Data_for_Portfolio['Net Margin % params'] = margin_params_list

            
            
            
#             print('tut2')  
            
            

            Data_for_Portfolio['Div Yield'] = valuation_and_quality_df['Buyback Yield %'] + valuation_ratios_df['Dividend Yield %']

            div_yield_list = [0]
            book_value_per_share_list = [0]
            dividend_payout_ratio_list = [0]

        #  (500-400)/400*100=25%

            for year_div in range(len(Data_for_Portfolio)):

                try:
                    div_yield_list.append((Data_for_Portfolio['Div Yield'][year_div+1] - Data_for_Portfolio['Div Yield'][year_div])/ Data_for_Portfolio['Div Yield'][year_div]*100)
                except:
                    pass
                try:
                    book_value_per_share_list.append((per_share_data_df['Book Value per Share'][year_div+1] - per_share_data_df['Book Value per Share'][year_div])/ per_share_data_df['Book Value per Share'][year_div]*100)
                except:
                    pass

                try:
                    dividend_payout_ratio_list.append((common_size_ratios_df['Dividend Payout Ratio'][year_div+1] - common_size_ratios_df['Dividend Payout Ratio'][year_div])/ common_size_ratios_df['Dividend Payout Ratio'][year_div]*100)
                except:
                    pass


            div_yield_list_5y = [0, 0, 0, 0, 0]
            book_value_per_share_list_5y = [0, 0, 0, 0, 0]
            dividend_payout_ratio_list_5y = [0, 0, 0, 0, 0]


            for year_div_5 in range(len(Data_for_Portfolio)):
                if len(total_equity_grows_list[year_div_5:5+year_div_5])  == 5:
                    mean_total_equity_grows_list.append(np.mean(total_equity_grows_list[year_div_5:5+year_div_5]))
                else:
                    pass

                if len(EPS_grows_list[year_div_5:5+year_div_5])  == 5:
                    mean_EPS_grows_list.append(np.mean(EPS_grows_list[year_div_5:5+year_div_5]))
                else:
                    pass

                if len(rvenue_grows_list[year_div_5:5+year_div_5])  == 5:
                    mean_rvenue_grows_list.append(np.mean(dividend_payout_ratio_list[year_div_5:5+year_div_5]))
                else:
                    pass


            Data_for_Portfolio['Book Value per Share 5Y'] = book_value_per_share_list
            Data_for_Portfolio['Div Yield 5Y'] = div_yield_list
            Data_for_Portfolio['Dividend Payout Ratio 5Y'] = dividend_payout_ratio_list

#             print('tut2')

    #             regression = pd.ols(y=Data_for_Portfolio['Net Margin %'], x=date_list)
    #             print(regression.summary)

    #             Data_for_Portfolio['Total Equity Grows'] = total_equity_grows_list
    #             Data_for_Portfolio['EPS without NRI Grows'] = EPS_grows_list
    #             Data_for_Portfolio['Revenue Grows'] = rvenue_grows_list
    #             Data_for_Portfolio['Net Margin %'] = income_df['Net Margin %']


            Data_for_Portfolio['Company'] = ticker
            Data_for_Portfolio['Date'] = list(date_list)

            Data_for_Portfolio = Data_for_Portfolio.replace([np.inf, -np.inf], 0)
            Data_for_Portfolio = Data_for_Portfolio.set_index('Company')
            Data_for_Portfolio = Data_for_Portfolio[::-1].fillna(0)

            sumz_frame= [Data_for_Portfolio, Data_for_Portfolio_TOTAL]
            Data_for_Portfolio_TOTAL = pd.concat(sumz_frame)

        except:
            print(ticker)
            pass

#         print(list(date_list))
# list(set(Data_for_Portfolio_TOTAL.index.tolist()))

<ipython-input-3-9cb4da001a8e>:204: RuntimeWarning: invalid value encountered in double_scalars
  div_yield_list.append((Data_for_Portfolio['Div Yield'][year_div+1] - Data_for_Portfolio['Div Yield'][year_div])/ Data_for_Portfolio['Div Yield'][year_div]*100)
<ipython-input-3-9cb4da001a8e>:213: RuntimeWarning: invalid value encountered in double_scalars
  dividend_payout_ratio_list.append((common_size_ratios_df['Dividend Payout Ratio'][year_div+1] - common_size_ratios_df['Dividend Payout Ratio'][year_div])/ common_size_ratios_df['Dividend Payout Ratio'][year_div]*100)
<ipython-input-3-9cb4da001a8e>:213: RuntimeWarning: divide by zero encountered in double_scalars
  dividend_payout_ratio_list.append((common_size_ratios_df['Dividend Payout Ratio'][year_div+1] - common_size_ratios_df['Dividend Payout Ratio'][year_div])/ common_size_ratios_df['Dividend Payout Ratio'][year_div]*100)
<ipython-input-3-9cb4da001a8e>:204: RuntimeWarning: divide by zero encountered in double_scalars
  div_yield_li

SGX:K6S


<ipython-input-3-9cb4da001a8e>:126: RuntimeWarning: divide by zero encountered in double_scalars
  EPS_grows_list.append((per_share_data_df['EPS without NRI'][year+1] - per_share_data_df['EPS without NRI'][year])/ per_share_data_df['EPS without NRI'][year]*100)
<ipython-input-3-9cb4da001a8e>:208: RuntimeWarning: divide by zero encountered in double_scalars
  book_value_per_share_list.append((per_share_data_df['Book Value per Share'][year_div+1] - per_share_data_df['Book Value per Share'][year_div])/ per_share_data_df['Book Value per Share'][year_div]*100)
<ipython-input-3-9cb4da001a8e>:130: RuntimeWarning: divide by zero encountered in double_scalars
  rvenue_grows_list.append((income_df['Revenue'][year+1] - income_df['Revenue'][year])/ income_df['Revenue'][year]*100)


SGX:G07


<ipython-input-3-9cb4da001a8e>:122: RuntimeWarning: invalid value encountered in double_scalars
  total_equity_grows_list.append((balance_df['Total Equity'][year+1] - balance_df['Total Equity'][year])/ balance_df['Total Equity'][year]*100)
<ipython-input-3-9cb4da001a8e>:122: RuntimeWarning: divide by zero encountered in double_scalars
  total_equity_grows_list.append((balance_df['Total Equity'][year+1] - balance_df['Total Equity'][year])/ balance_df['Total Equity'][year]*100)
<ipython-input-3-9cb4da001a8e>:208: RuntimeWarning: invalid value encountered in double_scalars
  book_value_per_share_list.append((per_share_data_df['Book Value per Share'][year_div+1] - per_share_data_df['Book Value per Share'][year_div])/ per_share_data_df['Book Value per Share'][year_div]*100)
<ipython-input-3-9cb4da001a8e>:126: RuntimeWarning: invalid value encountered in double_scalars
  EPS_grows_list.append((per_share_data_df['EPS without NRI'][year+1] - per_share_data_df['EPS without NRI'][year])/ per_sha

SGX:JYEU
SGX:U13
SGX:MXNU


In [4]:
len(list(set(Data_for_Portfolio_TOTAL.index.tolist())))

195

In [5]:
# list(set(Data_for_Portfolio_TOTAL.index.tolist()))

In [6]:
# # # ['annuals']

# tickers = si.tickers_sp500()

# Data_for_Portfolio_TOTAL = pd.DataFrame()



# for ticker in tickers:
#     with open(f'BANKA/data_json_{ticker}.pickle', 'rb') as f:
#         data_json = pickle.load(f)

#     with open(f'BANKA/data_json_keyratios_{ticker}.pickle', 'rb') as f:
#         data_json_keyratios = pickle.load(f)

# #     print(data_json)    
#     try:
#         date_list = pd.Series(data_json['financials']['annuals']['Fiscal Year'])
#         keyratios = pd.DataFrame(data_json_keyratios['Fundamental'], index=[0]) 
#         income_df = pd.DataFrame(data_json['financials']['annuals']['income_statement']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('-', 0).replace('', 0).replace('N/A', 0).astype(float)
#         balance_df = pd.DataFrame(data_json['financials']['annuals']['balance_sheet']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('', 0).replace('-', 0).replace('N/A', 0).astype(float)
#         cashflow_df = pd.DataFrame(data_json['financials']['annuals']['cashflow_statement']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('', 0).replace('-', 0).replace('N/A', 0).astype(float)
#         valuation_ratios_df = pd.DataFrame(data_json['financials']['annuals']['valuation_ratios']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('', 0).replace('-', 0).replace('N/A', 0).astype(float)
#         valuation_and_quality_df = pd.DataFrame(data_json['financials']['annuals']['valuation_and_quality']).set_index(date_list).drop(['Restated Filing Date', 'Filing Date', 'Earnings Release Date'], axis=1).replace('', 0).replace('No Debt', 0).replace('At Loss', 0).replace('-', 0).replace('N/A', 0).astype(float)
#         common_size_ratios_df = pd.DataFrame(data_json['financials']['annuals']['common_size_ratios']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('', 0).replace('-', 0).replace('N/A', 0).replace('Negative Tangible Equity', 0).astype(float)
#         # per_share_data_array_df = pd.DataFrame(data_json['financials']['annuals']['per_share_data_array']).set_index(date_list).replace('-', 0).replace('N/A', 0).astype(float)
#         per_share_data_df = pd.DataFrame(data_json['financials']['annuals']['per_share_data_array']).set_index(date_list).replace('', 0).replace('No Debt', 0).replace('-', 0).replace('N/A', 0).astype(float)
        
#         check = 1
#     except:
#         pass
            
#     if check == 1:
    
#         Data_for_Portfolio = pd.DataFrame()
#     #     
#         Data_for_Portfolio['E/P'] = income_df['Net Income'] / valuation_and_quality_df['Market Cap'] 
#         # income_df['Net Income']
#         # valuation_and_quality_df.replace('-', 0)
#         Data_for_Portfolio
#         try:
#             Data_for_Portfolio['EBITDA/EV'] = income_df['EBITDA'] / (valuation_and_quality_df['Enterprise Value ($M)']*1000000)
#         except:
#             Data_for_Portfolio['EBITDA/EV'] = Data_for_Portfolio['E/P'] * 0

#         Data_for_Portfolio['ncfcommon'] = cashflow_df['Free Cash Flow']/valuation_and_quality_df['Shares Outstanding (EOP)']
#         try:
#             Data_for_Portfolio['Total Debt'] = (balance_df['Short-Term Debt'] + balance_df['Long-Term Debt'])
#         except:
#             Data_for_Portfolio['Total Debt'] = Data_for_Portfolio['ncfcommon'] * 0
# #             print(Data_for_Portfolio)

#         Data_for_Portfolio['Shareholder Yield'] = -((Data_for_Portfolio['Total Debt'] + cashflow_df['Free Cash Flow'] \
#                                          + Data_for_Portfolio['ncfcommon']) / valuation_and_quality_df['Market Cap'])


#         Data_for_Portfolio['FCF/P'] = cashflow_df['Free Cash Flow'] / valuation_and_quality_df['Market Cap']
#         Data_for_Portfolio
#         # Data_for_Portfolio['Shareholder Yield'] 
#         Data_for_Portfolio['Book Value per Share'] = per_share_data_df['Book Value per Share']
#         Data_for_Portfolio['Dividends per Share'] =per_share_data_df['Dividends per Share'] 
#         Data_for_Portfolio['Dividend Payout Ratio'] =common_size_ratios_df['Dividend Payout Ratio']

#         try:
#             Data_for_Portfolio['FCF/Assets'] = cashflow_df['Free Cash Flow'] / balance_df['Total Current Assets']
#         except:
#             Data_for_Portfolio['FCF/Assets'] = cashflow_df['Free Cash Flow'] / balance_df['Total Assets']
#         #Can you generate returns on investment?   



#         Data_for_Portfolio['ROA'] = common_size_ratios_df['ROA %']  
#         Data_for_Portfolio['ROIC'] = common_size_ratios_df['ROIC %']
#         #Do you have a defendable business model?
#         try:
#             Data_for_Portfolio['GROSS MARGIN'] = common_size_ratios_df['Gross Margin %']
#         except:
#             Data_for_Portfolio['GROSS MARGIN'] = Data_for_Portfolio['ROA'] * 0
#         #Current Financial Strength
#         try:
#             Data_for_Portfolio['CURRENT RATIO'] = balance_df['Total Current Assets'] / balance_df['Total Current Liabilities']
#         except:
#             Data_for_Portfolio['CURRENT RATIO'] = balance_df['Total Assets'] / balance_df['Total Liabilities']


#         try:
#             Data_for_Portfolio['INTEREST/EBITDA'] = income_df['Interest Expense'] / income_df['EBITDA']
#         except:
#             try:
#                 Data_for_Portfolio['INTEREST/EBITDA'] = income_df['Interest Expense (Positive)'] / income_df['EBITDA']
#             except:
#                 Data_for_Portfolio['INTEREST/EBITDA'] = income_df['Interest Expense'] / 0




#         Data_for_Portfolio['Company'] = ticker
#         Data_for_Portfolio['Date'] = list(date_list)

#         Data_for_Portfolio = Data_for_Portfolio.replace([np.inf, -np.inf], 0)
#         Data_for_Portfolio = Data_for_Portfolio.set_index('Company')
#         Data_for_Portfolio = Data_for_Portfolio[::-1]

#         sumz_frame= [Data_for_Portfolio, Data_for_Portfolio_TOTAL]
#         Data_for_Portfolio_TOTAL = pd.concat(sumz_frame)



# #         print(list(date_list))


In [7]:
# tickers = ['AAPL', 'INTC']

yahoo_ticker_list_full = []

for tic in tickers:
    yahoo_ticker_list_full.append(tic.replace(exchange,'')+exchange_yahoo )

price_yahoo_main = yf.download(yahoo_ticker_list_full)
price_yahoo_main = price_yahoo_main['Adj Close'].fillna(method='backfill')




years_len = int(cheked_year_end) - int(cheked_year)

portfolio_profit_final = []
index_profit_final = []
max_dd_list = []


Percentile_split = .2

Winsorize_Threshold = .025

# for i in range(years_len):
for i in range(years_len):
    print('i'*50)
    print(i)

    df_res = pd.DataFrame()
    
    for ticker in tickers:
        try:

            Data_for_Portfolio_tick = Data_for_Portfolio_TOTAL.loc[ticker].fillna(0).iloc[int(cheked_year_end) - (int(cheked_year)-1+i)]
#             print(Data_for_Portfolio_tick)
            sum_frame = [pd.DataFrame([Data_for_Portfolio_tick]), df_res]
            df_res = pd.concat(sum_frame )  
            
        except:
            pass
        
    
    Data_for_Portfolio_master_filter = df_res
    
    yahoo_ticker_list = []

    for tic in Data_for_Portfolio_master_filter.index.tolist():
        yahoo_ticker_list.append(tic.replace(exchange,'')+exchange_yahoo )
         

   #Winsorize the metric data and compress outliers if desired
    Data_for_Portfolio_master_filter['E/P Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['E/P'], \
                               limits=Winsorize_Threshold)
    Data_for_Portfolio_master_filter['EBITDA/EV Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['EBITDA/EV'], \
                               limits=Winsorize_Threshold)
    Data_for_Portfolio_master_filter['FCF/P Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['FCF/P'], \
                               limits=Winsorize_Threshold)

    #create Z score to normalize the metrics
    Data_for_Portfolio_master_filter['E/P Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['E/P Winsorized'])
    Data_for_Portfolio_master_filter['EBITDA/EV Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['EBITDA/EV Winsorized'])
    Data_for_Portfolio_master_filter['FCF/P Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['FCF/P Winsorized'])

    Data_for_Portfolio_master_filter['Valuation Score'] = \
            Data_for_Portfolio_master_filter['E/P Z score'] \
            + Data_for_Portfolio_master_filter['EBITDA/EV Z score']\
            + Data_for_Portfolio_master_filter['FCF/P Z score']

    Data_for_Portfolio_master_filter['FCF/P Winsorized']

    ###### QUALITY FACTOR ######  
    Data_for_Portfolio_master_filter = Data_for_Portfolio_master_filter.replace([np.inf, -np.inf], 0)


    Data_for_Portfolio_master_filter['FCF/Assets Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['FCF/Assets'], \
                               limits=Winsorize_Threshold)
    Data_for_Portfolio_master_filter['ROA Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['ROA'], \
                               limits=Winsorize_Threshold)
    Data_for_Portfolio_master_filter['ROIC Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['ROIC'], \
                                limits=Winsorize_Threshold)
    Data_for_Portfolio_master_filter['Gross Margin Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['GROSS MARGIN'], \
                               limits=Winsorize_Threshold)
    Data_for_Portfolio_master_filter['Current Ratio Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['CURRENT RATIO'], \
                               limits=Winsorize_Threshold)
    Data_for_Portfolio_master_filter['Interest/EBITDA Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['INTEREST/EBITDA'], \
                               limits=Winsorize_Threshold)

    #create Z score
    Data_for_Portfolio_master_filter = Data_for_Portfolio_master_filter.replace([np.inf, -np.inf], 0)


    Data_for_Portfolio_master_filter['FCF/Assets Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['FCF/Assets Winsorized'])
    Data_for_Portfolio_master_filter['ROA Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['ROA Winsorized'])
    Data_for_Portfolio_master_filter['ROIC Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['ROIC Winsorized'])
    Data_for_Portfolio_master_filter['Gross Margin Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['Gross Margin Winsorized'])
    Data_for_Portfolio_master_filter['Current Ratio Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['Current Ratio Winsorized'])
    Data_for_Portfolio_master_filter['Interest/EBITDA Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['Interest/EBITDA Winsorized'])

    Data_for_Portfolio_master_filter['Quality Score'] = \
        Data_for_Portfolio_master_filter['FCF/Assets Z score'] \
            + Data_for_Portfolio_master_filter['ROA Z score'] \
            + Data_for_Portfolio_master_filter['ROIC Z score']\
            + Data_for_Portfolio_master_filter['Gross Margin Z score']\
            + Data_for_Portfolio_master_filter['Current Ratio Z score']\
            - Data_for_Portfolio_master_filter['Interest/EBITDA Z score']


    
   
    
    ###### SHAREHOLDER YIELD FACTOR #####

    Data_for_Portfolio_master_filter = Data_for_Portfolio_master_filter.replace([np.inf, -np.inf], 0)

    Data_for_Portfolio_master_filter['Shareholder Yield Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['Shareholder Yield'], \
                               limits=Winsorize_Threshold)
    Data_for_Portfolio_master_filter['Shareholder Yield Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['Shareholder Yield Winsorized'])
    Data_for_Portfolio_master_filter['Shareholder Yield Score'] = \
        Data_for_Portfolio_master_filter['Shareholder Yield Z score'] 
        
        
        ###### LOW VOLATILITY FACTOR ######

    #must have fundamental data from previous factors for price based factors
    #as some equities have price data and no fundamental data which should not
    #be included

    # treasury = 'RGBI.ME'
    start = cheked_year
    # end = current_date
    end =  cheked_year_end

#     price_yahoo = yf.download(Data_for_Portfolio_master_filter.index.tolist())
    price_yahoo = price_yahoo_main[yahoo_ticker_list]

    Sector_stock_returns =  price_yahoo.pct_change()      

    #create rolling vol metric for previous 2 years
    Sector_stock_rolling_vol = Sector_stock_returns.rolling(252*2).std()

    #Choose second to last trading day to look at previous vol   
    #Sometimes the dates are off when trying to line up end of quarter and business
    #days so to eliminate errors in the for loop I go to day of quarter, shift forward
    #a business day and then go back two business days

    

    Filter_Vol_Signal = Sector_stock_rolling_vol[str(int(start)+i)].dropna()
    
    

    # Filter_Vol_Signal_Sort = Filter_Vol_Signal.sort_values().dropna() # для цикла паска тикеров

    #create z score and rank for the Volatility Factor
    # frame = { 'Vol': Filter_Vol_Signal} 

    # Filter_Vol_Signal_df = pd.DataFrame(frame)
    
    
#     print(pd.DataFrame(stats.zscore(Filter_Vol_Signal)).mean())
#     print(pd.DataFrame(stats.zscore(Filter_Vol_Signal)))

#     Filter_Vol_Signal = Filter_Vol_Signal.fillna(0)

    # Filter_Vol_Signal_df['Vol Z Score'] = stats.zscore(Filter_Vol_Signal) 
    # Filter_Vol_Signal_df = Filter_Vol_Signal_df.reset_index()
    # print(Filter_Vol_Signal_df)

    
    Data_for_Portfolio_master_filter['Vol Z Score'] = pd.DataFrame(stats.zscore(Filter_Vol_Signal)).fillna(0).mean().tolist()
#     print(stats.zscore(Filter_Vol_Signal))
    # Data_for_Portfolio_master_filter = Data_for_Portfolio_master_filter.merge(Filter_Vol_Signal_df, how = 'inner', on = ['ticker']) 

    
    
        ###### NEWWWWWWWWWWWWWWWWWWWWWWWWWWWW FACTOR #####
   
    Data_for_Portfolio_master_filter['Total Equity Grows 5Y Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['Total Equity Grows 5Y'], \
                               limits=Winsorize_Threshold)
    Data_for_Portfolio_master_filter['EPS without NRI Grows 5Y Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['EPS without NRI Grows 5Y'], \
                               limits=Winsorize_Threshold)
    Data_for_Portfolio_master_filter['Revenue Grows 5Y Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['Revenue Grows 5Y'], \
                                limits=Winsorize_Threshold)


    
    Data_for_Portfolio_master_filter['Total Equity Grows 5Y Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['Total Equity Grows 5Y Winsorized'])
    Data_for_Portfolio_master_filter['EPS without NRI Grows 5Y Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['EPS without NRI Grows 5Y Winsorized'])
    Data_for_Portfolio_master_filter['Revenue Grows 5Y Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['Revenue Grows 5Y Winsorized'])
    
    

    Data_for_Portfolio_master_filter['Net Margin % params score'] = np.where(Data_for_Portfolio_master_filter['Net Margin % params'] > 0, 1,0)
    
    
    Data_for_Portfolio_master_filter['Grows score'] = Data_for_Portfolio_master_filter['Net Margin % params score']+ \
                                                    Data_for_Portfolio_master_filter['Total Equity Grows 5Y Z score']+ \
                                                    Data_for_Portfolio_master_filter['EPS without NRI Grows 5Y Z score']+ \
                                                    Data_for_Portfolio_master_filter['Revenue Grows 5Y Z score']
    
    
    
    
    
    ###### TREND FACTOR #####
    
    total_trend_score = []
    
    for tic in yahoo_ticker_list:
        try:
            df = yf.download(tic, str(int(start)+i-2)+'-1-1', str(int(start)+i+1)+'-1-1')
        #     print(df)
            df = df[['Open', 'High', 'Low', 'Adj Close']]
            df['open'] = df['Open'].shift(1)
            df['high'] = df['High'].shift(1)
            df['low'] = df['Low'].shift(1)
            df['close'] = df['Adj Close'].shift(1)

            df = df[['open', 'high', 'low', 'close']]
            df = df.dropna()

            unsup = mix.GaussianMixture(n_components=4,
                                        covariance_type="spherical",
                                        n_init=100,
                                        random_state=42)
            unsup.fit(np.reshape(df, (-1, df.shape[1])))
            regime = unsup.predict(np.reshape(df, (-1, df.shape[1])))
            df['Return'] = np.log(df['close'] / df['close'].shift(1))
            Regimes = pd.DataFrame(regime, columns=['Regime'], index=df.index) \
                .join(df, how='inner') \
                .assign(market_cu_return=df.Return.cumsum()) \
                .reset_index(drop=False) \
                .rename(columns={'index': 'Date'})

            order = [0, 1, 2, 3]
        #     fig = sns.FacetGrid(data=Regimes, hue='Regime', hue_order=order, aspect=2, height=4)
        #     fig.map(plt.scatter, 'Date', 'market_cu_return', s=4).add_legend()
        #     plt.show()

            mean_for_regime = []
            cur_price = df['close'][-1]

            total_position = 0

            for j in order:
                mean_for_regime.append(unsup.means_[j][0])
    #             print('Mean for regime %i: '%i,unsup.means_[i][0])
        #         print('Co-Variance for regime %i: '%i,(unsup.covariances_[i]))

            mean_for_regime = np.sort(mean_for_regime)   
            for val in  mean_for_regime:
                if cur_price > val:
                    total_position += 0.25
                else:
                    pass
                
            total_trend_score.append(total_position)
    #         print(mean_for_regime)
    #         print('cur_price')
    #         print(cur_price)
    #         print('total_position')
    #         print(total_position)
        except:
            total_trend_score.append(0)

        
    


    Data_for_Portfolio_master_filter['Trend Score'] = total_trend_score
    
    #This is a very simply way to see how much a stock is in a trend up or down
    #You could easily make this more complex/robust but it would cost you in 
    #execution time
#     df_sma_50 = price_yahoo.rolling(50).mean()
#     df_sma_100 = price_yahoo.rolling(100).mean()
#     df_sma_150 = price_yahoo.rolling(150).mean()
#     df_sma_200 = price_yahoo.rolling(200).mean()


#     Filter_Trend_Signal_50 = df_sma_50[str(int(cheked_year)+i)]
#     Filter_Trend_Signal_100 = df_sma_100[str(int(cheked_year)+i)]
#     Filter_Trend_Signal_150 = df_sma_150[str(int(cheked_year)+i)]
#     Filter_Trend_Signal_200 = df_sma_200[str(int(cheked_year)+i)]

#     Price_Signal = price_yahoo[str(int(cheked_year)+i)]

#     SMA_all = pd.DataFrame()
#     SMA_50 = pd.DataFrame(np.where(Price_Signal > Filter_Trend_Signal_50,1,0)).mean()
#     SMA_100 = pd.DataFrame(np.where(Price_Signal > Filter_Trend_Signal_100,1,0)).mean()
#     SMA_150 = pd.DataFrame(np.where(Price_Signal > Filter_Trend_Signal_150,1,0)).mean()
#     SMA_200 = pd.DataFrame(np.where(Price_Signal > Filter_Trend_Signal_200,1,0)).mean()

#     SMA_all['SMA_50'] = SMA_50
#     SMA_all['SMA_100'] = SMA_100
#     SMA_all['SMA_150'] = SMA_150
#     SMA_all['SMA_200'] = SMA_200
#     SMA_all['Trend Score'] = np.mean(SMA_all, axis=1)

#     # print(SMA_all)
#     Data_for_Portfolio_master_filter['Trend Score'] = np.mean(SMA_all, axis=1).tolist()


    
    
    

    ###### MOMENTUM FACTOR #####

#     print('tut')
    
    # tickers_momentum = list(Sector_stock_prices_vol_df_1_wide.columns)
    #from the academic literature of 12 months - 1 month momentum 
#     df_mom_11_months = price_yahoo[str(int(cheked_year)+i)].pct_change(22*11)
#     Data_for_Portfolio_master_filter['Momentum Score'] = pd.DataFrame(stats.zscore(df_mom_11_months.iloc[242:])).fillna(0).mean().tolist()
#     # Data_for_Portfolio_master_filter = Data_for_Portfolio_master_filter.merge(Filter_MOM_df[['ticker','Momentum Score']], how = 'inner', on = ['ticker'])
#     Data_for_Portfolio_master_filter



    prices = price_yahoo_main[yahoo_ticker_list].asfreq('BM')
    prices_yearly_returns = prices.pct_change(12)
    prices_yearly_signal = np.where(prices_yearly_returns[str(int(cheked_year)+i)].iloc[-1] > 0, 1, 0)
    Data_for_Portfolio_master_filter['Momentum Score'] = prices_yearly_signal

    
       
    ### Create Composite Score from factors ###

    #Because we made all the factors with a z score each factor should have equal
    #weight in the composite. You could consider changing the weights based on 
    #historical statistical significance or whatever else seems reasonable

    #This particular scoring system only invests in companies with 
    #positive trend/momentum after ranking by the other factors

    Data_for_Portfolio_master_filter['Total Score'] = Data_for_Portfolio_master_filter['Valuation Score'] +  \
        Data_for_Portfolio_master_filter['Quality Score'] + Data_for_Portfolio_master_filter['Shareholder Yield Score'] +\
        Data_for_Portfolio_master_filter['Momentum Score'] + Data_for_Portfolio_master_filter['Trend Score'] + \
        Data_for_Portfolio_master_filter['Grows score']
    
    
    
    Data_for_Portfolio_master_filter = Data_for_Portfolio_master_filter.sort_values('Total Score', ascending=False)
    
    top_rated_company = Data_for_Portfolio_master_filter[:int(len(Data_for_Portfolio_master_filter) \
                                                              * Percentile_split)].index.tolist()
    top_rated_company

    low_rated_company = Data_for_Portfolio_master_filter[-int(len(Data_for_Portfolio_master_filter) \
                                                              * Percentile_split):].index.tolist()
    low_rated_company
    
   
    
    
    
    start_hayoo = str(int(start)+i+1)+'-1-1'
    end_hayoo = str(int(start)+i+2)+'-1-1'
    
    cum_str_returns_bh = (price_yahoo[yahoo_ticker_list][str(int(start)+i+1)].fillna(method='backfill').pct_change() + 1).cumprod()
    running_max_BH = np.maximum.accumulate(cum_str_returns_bh[1:].fillna(method='backfill') )
    drawdown_BH = (cum_str_returns_bh[1:])/running_max_BH - 1
    max_dd = drawdown_BH.min()*100


#     try:
#         Data_for_Portfolio_master_filter['Max DD'] = max_dd.values 
#     except:
#         Data_for_Portfolio_master_filter['Max DD'] = [max_dd]
    
    max_dd_list.append(max_dd.min())
    
#     # == Доходность
    
    portfolio_profit = [] 
    profit_list_index = 0

    top_rated_company_yahoo = []
    low_rated_company_yahoo = []
    
    
    for tic in top_rated_company:
        top_rated_company_yahoo.append(tic.replace(exchange,'')+exchange_yahoo )
        
    for tic in low_rated_company:
        low_rated_company_yahoo.append(tic.replace(exchange,'')+exchange_yahoo )
        
  
    
#     profit_yah = yf.download(top_rated_company, start_hayoo, end_hayoo)['Adj Close'].fillna(method='backfill') 
    profit_yah = price_yahoo_main[top_rated_company_yahoo][str(int(start)+i+1)].fillna(method='backfill') 
    profit = (profit_yah.iloc[-1]-profit_yah.iloc[0])/profit_yah.iloc[0]
    profit = profit.replace([np.inf, -np.inf], np.nan).dropna()
    portfolio_profit = profit.values.tolist()
#     print(profit_yah)
            
    
#     for company in top_rated_company:
# #         print('tut1')
#         try:
#             profit_yah = yf.download(company, start_hayoo, end_hayoo)['Adj Close'].fillna(method='backfill') 
#             profit = (profit_yah[-1]-profit_yah[0])/profit_yah[0]
#             (1 + profit).cumprod()[-1]
#             portfolio_profit.append(profit)
#             print(profit)
#         except:
#             pass
#     portfolio_profit


#     profit_list_index = 0

    profit_yah_index = yf.download(index)['Adj Close'].fillna(method='backfill')[str(int(start)+i)] 
    profit_index = (profit_yah_index[-1]-profit_yah_index[0])/profit_yah_index[0]

    

    print('Год начальный')
    print(start_hayoo)
    
#     profit_list_index = profit_index

    portfolio_profit_final.append(np.mean(portfolio_profit)*100)
    index_profit_final.append(profit_index*100)

#     returnezzz = pd.DataFrame()
#     returnezzz['Portfolio'] = [np.mean(portfolio_profit)*100]
#     returnezzz['Index'] = [profit_list_index*100]

    print(portfolio_profit_final)

    print('top_rated_company')
    print(top_rated_company)
    print('low_rated_company')
    print(low_rated_company)
       

    print('Max DD')
    print(max_dd_list)
    print(np.min(max_dd_list))




[*********************100%***********************]  200 of 200 completed

1 Failed download:
- ADQU.SI: No data found, symbol may be delisted
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
0


<ipython-input-7-8040e5107bdb>:169: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  Filter_Vol_Signal = Sector_stock_rolling_vol[str(int(start)+i)].dropna()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- CLN.SI: Data doesn't exist for startDate = 1356987600, endDate = 1451595600
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[****

C:\Users\Anton\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- CJLU.SI: Data doesn't exist for startDate = 1356987600, endDate = 1451595600
[***

...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TCU.SI: Data doesn't exist for startDate = 1483218000, endDate = 1577826000
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[****

C:\Users\Anton\anaconda3\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\Anton\anaconda3\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\Anton\anaconda3\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\Anton\anaconda3\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  label =

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

<ipython-input-7-8040e5107bdb>:350: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  prices_yearly_signal = np.where(prices_yearly_returns[str(int(cheked_year)+i)].iloc[-1] > 0, 1, 0)
<ipython-input-7-8040e5107bdb>:388: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  cum_str_returns_bh = (price_yahoo[yahoo_ticker_list][str(int(start)+i+1)].fillna(method='backfill').pct_change() + 1).cumprod()
<ipython-input-7-8040e5107bdb>:419: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  profit_yah = price_y

[*********************100%***********************]  1 of 1 completed
Год начальный
2020-1-1
[49.11992688307967, 34.031798019321634, -8.901456292050778, 23.770800242592877, 1.330804288611202]
top_rated_company
['SGX:AWX', 'SGX:C41', 'SGX:OYY', 'SGX:1D0', 'SGX:QES', 'SGX:5WE', 'SGX:5WH', 'SGX:5CP', 'SGX:UD1U', 'SGX:CHZ', 'SGX:S68', 'SGX:AGS', 'SGX:N2IU', 'SGX:5G3', 'SGX:5DD', 'SGX:ME8U', 'SGX:H02', 'SGX:CY6U', 'SGX:S85', 'SGX:K3RD', 'SGX:VL6', 'SGX:F03', 'SGX:J91U', 'SGX:A26', 'SGX:J69U', 'SGX:B61', 'SGX:O39', 'SGX:L09', 'SGX:K2LU', 'SGX:BS6', 'SGX:Q5T', 'SGX:OU8', 'SGX:C2PU', 'SGX:A50', 'SGX:M30', 'SGX:E28', 'SGX:TS0U', 'SGX:LJ3', 'SGX:O5RU']
low_rated_company
['SGX:P34', 'SGX:K3DD', 'SGX:T15', 'SGX:B28', 'SGX:E5H', 'SGX:K3HD', 'SGX:BHK', 'SGX:5JK', 'SGX:BN4', 'SGX:C29', 'SGX:S07', 'SGX:M01', 'SGX:P9D', 'SGX:O10', 'SGX:BDX', 'SGX:5UX', 'SGX:STG', 'SGX:T24', 'SGX:G41', 'SGX:8K7', 'SGX:5EN', 'SGX:U96', 'SGX:K3FD', 'SGX:579', 'SGX:F1E', 'SGX:D03', 'SGX:XZL', 'SGX:NS8U', 'SGX:NC2', 'SGX:F83

In [8]:
returnez_cum_port = pd.DataFrame(portfolio_profit_final).dropna()  
returnez_cum_index = pd.DataFrame(index_profit_final).dropna()  

returnez = pd.DataFrame()

returnez['Страна'] = [LIST]
returnez['Начало периода'] = [cheked_year]
returnez['Дходность с ребалансировкой портфеля'] = ((1 + (returnez_cum_port/100)).cumprod().iloc[-1]-1)*100
returnez['Дходность Индекса'] = ((1 + (returnez_cum_index/100)).cumprod().iloc[-1]-1)*100
returnez['Max DD'] = [np.min(max_dd_list)]

# gc = gd.service_account(filename='Seetzzz-1cb93f64d8d7.json')
# worksheet = gc.open("Тесты бэктестинга").worksheet('Мульти-фактор2')

# worksheet.update('A20', [returnez.columns.tolist()] + returnez.values.tolist())

returnez

,Страна,Начало периода,Дходность с ребалансировкой портфеля,Дходность Индекса,Max DD
0,Сингапур,2015,128.357168,24.126498,-108.052548


In [9]:
profit

AWX.SI     0.709581
C41.SI     0.361834
OYY.SI     0.579740
1D0.SI     0.371609
QES.SI    -0.041457
5WE.SI    -0.090909
5WH.SI    -0.265000
5CP.SI    -0.347877
UD1U.SI   -0.149523
CHZ.SI    -0.071400
S68.SI     0.090820
AGS.SI     0.059247
N2IU.SI   -0.076036
5G3.SI    -0.111468
5DD.SI     0.438282
ME8U.SI    0.147130
H02.SI    -0.117722
CY6U.SI   -0.062110
S85.SI    -0.138951
K3RD.SI    0.000000
VL6.SI    -0.080855
F03.SI     0.012017
J91U.SI   -0.209111
A26.SI    -0.178839
J69U.SI   -0.081118
B61.SI    -0.143909
O39.SI    -0.026714
L09.SI     0.591614
K2LU.SI   -0.065435
BS6.SI    -0.142444
Q5T.SI    -0.150619
OU8.SI    -0.209979
C2PU.SI    0.195653
A50.SI    -0.253731
E28.SI     0.482207
TS0U.SI   -0.301928
LJ3.SI    -0.139333
O5RU.SI   -0.077560
dtype: float64